In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = (20.0, 20.0)

In [ ]:
from tda.experiments.mahalanobis.mahalanobis_binary import Config, run_experiment, get_feature_datasets, evaluate_detector, \
    compute_means_and_sigmas_inv, get_config, logger
from tda.models.architectures import mnist_lenet, get_architecture, mnist_mlp
from tda.models.datasets import Dataset
from tda.models import get_deep_model

In [ ]:
config = Config(
    # Number of epochs for the model
    epochs = 50,
    # Dataset we consider (MNIST, SVHN)
    dataset = "MNIST",
    # Name of the architecture
    architecture = mnist_lenet.name,
    # Size of the dataset used for the experiment
    dataset_size = 100,
    # Type of attack (FGSM, BIM, CW)
    attack_type = "FGSM",
    # Epsilon for the preprocessing step (see the paper)
    preproc_epsilon = 0.01,
    # Noise to consider for the noisy samples
    noise = 0.0,
    # Number of sample per class to estimate mu_class and sigma_class
    number_of_samples_for_mu_sigma = 100
)

In [ ]:
dataset = Dataset(name=config.dataset)

architecture = get_deep_model(
    num_epochs=config.epochs,
    dataset=dataset,
    architecture=get_architecture(config.architecture),
    train_noise=0.0
)

mean_per_class, sigma_per_class_inv = compute_means_and_sigmas_inv(
    config=config,
    dataset=dataset,
    architecture=architecture
)

In [ ]:
fig, axs = plt.subplots(len(sigma_per_class_inv)//2+1, 2)
i = 0

for layer_idx in sigma_per_class_inv:
    px = i // 2
    py = i % 2
    axs[px][py].imshow(sigma_per_class_inv[layer_idx])
    i+=1

In [ ]:
datasets = dict()
for epsilon in [0.01, 0.025, 0.05, 0.1, 0.4, 1.0]:
    ds_train, ds_test = get_feature_datasets(
        config=config,
        epsilon=epsilon,
        dataset=dataset,
        architecture=architecture,
        mean_per_class=mean_per_class,
        sigma_per_class_inv=sigma_per_class_inv
    )
    
    datasets[epsilon] = (ds_train, ds_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
import numpy as np

aucs = dict()
coefs = dict() 

for epsilon in datasets:
    
    ds_train, ds_test = datasets[epsilon]
    
    typ_detector = "SVM"
    
    if typ_detector == "LR":
        detector = LogisticRegression(
            fit_intercept=True,
            verbose=0,
            tol=1e-2,
            max_iter=10000,
            solver='lbfgs',
            penalty='none',
            C=1.0
        )
    elif typ_detector == "SVM":
        detector = SVC(
            tol=1e-5,
            C=1.0,
            kernel="rbf",
            verbose=False,
            max_iter=10000,
            gamma='scale'
        )
    else:
        raise NotImplementedError(f"Unknown detector method {typ_detector}")

    detector.fit(X=ds_train.iloc[:, :-1], y=ds_train.iloc[:, -1])
    #coef = list(detector.coef_.flatten())
    #logger.info(f"Coefs of detector {coef}")
    #logger.info(f"Intercept of detector {detector.intercept_}")
    train_predictions = detector.predict(X=ds_train.iloc[:, :-1])
    
    test_predictions = detector.predict(X=ds_test.iloc[:, :-1]) # [:, 1]
    logger.info(f"Average prediction {np.mean(test_predictions)}")
    auc = roc_auc_score(y_true=ds_test.iloc[:, -1], y_score=test_predictions)
    
    logger.info(f"AUC for {epsilon} => {auc}")

    aucs[epsilon] = auc
    #coefs[epsilon] = coef
    
logger.info(f"All AUCS are {aucs}")